In [2]:
def read_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('nsmc-master/ratings_train.txt')
test_data = read_data('nsmc-master/ratings_test.txt')

In [3]:
from konlpy.tag import Okt

okt = Okt()

D:\anaconda\envs\obj\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [4]:
import json
import os
from pprint import pprint

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json', encoding='utf-8') as f:
        train_docs = json.load(f)
    with open('test_docs.json', encoding='utf-8') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [5]:
tokens = [t for d in train_docs for t in d[0]]

In [6]:
import nltk
text = nltk.Text(tokens, name='NMSC')

In [7]:
selected_words = [f[0] for f in text.vocab().most_common(6000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [33]:
import pickle
with open('wdata.p', 'wb') as file:
    pickle.dump(selected_words, file)

In [24]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [29]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(6000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=100, batch_size=2048)
results = model.evaluate(x_test, y_test)

Train on 150000 samples
Epoch 1/100
150000/150000 [==============================] - 4s 25us/sample - loss: 0.4353 - binary_accuracy: 0.8223
Epoch 2/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.3421 - binary_accuracy: 0.8546
Epoch 3/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.3215 - binary_accuracy: 0.8632
Epoch 4/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.3028 - binary_accuracy: 0.8702
Epoch 5/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.2854 - binary_accuracy: 0.8787
Epoch 6/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.2678 - binary_accuracy: 0.8882
Epoch 7/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.2509 - binary_accuracy: 0.8969
Epoch 8/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.2335 - binary_accuracy: 0.9062
Epoch 9/100
150000/150000 [====================

150000/150000 [==============================] - 1s 9us/sample - loss: 0.0154 - binary_accuracy: 0.9928
Epoch 71/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0153 - binary_accuracy: 0.9926
Epoch 72/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0164 - binary_accuracy: 0.9926
Epoch 73/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0160 - binary_accuracy: 0.9928
Epoch 74/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0159 - binary_accuracy: 0.9926
Epoch 75/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0147 - binary_accuracy: 0.9928
Epoch 76/100
150000/150000 [==============================] - 1s 10us/sample - loss: 0.0155 - binary_accuracy: 0.9929
Epoch 77/100
150000/150000 [==============================] - 1s 9us/sample - loss: 0.0168 - binary_accuracy: 0.9925
Epoch 78/100
150000/150000 [==============================] - 1s 9us/sample 

In [30]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [31]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")

[올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.]는 100.00% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.]는 98.52% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ]는 100.00% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[믿고 보는 감독이지만 이번에는 아니네요]는 100.00% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[주연배우 때문에 봤어요]는 99.99% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;



In [28]:
with open('xtyt.p', 'wb') as file:
    pickle.dump(x_train, file)
    pickle.dump(x_test, file)
    pickle.dump(y_train, file)
    pickle.dump(y_test, file)

In [34]:
model.save('model.h5')